In [14]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from z3 import *

# Carregando o dataset MNIST e selecionando apenas as classes 0 e 1
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(np.int8)

# Selecionar apenas as classes 0 e 1
mask = (y == 0) | (y == 1)
X = X[mask]
y = y[mask]

# Verificando o número de amostras por classe
print("Número de amostras por classe após filtragem:")
print("Classe 0:", np.sum(y == 0))
print("Classe 1:", np.sum(y == 1))

# Reduzir para as primeiras 500 amostras de cada classe (se possível)
num_samples = 500
X_class_0 = X[y == 0].iloc[:num_samples].to_numpy()
y_class_0 = y[y == 0].iloc[:num_samples].to_numpy()
X_class_1 = X[y == 1].iloc[:num_samples].to_numpy()
y_class_1 = y[y == 1].iloc[:num_samples].to_numpy()

X = np.vstack((X_class_0, X_class_1))
y = np.hstack((y_class_0, y_class_1))
feature_names = [f'pixel_{i}' for i in range(X.shape[1])]  # Nomes das características (pixels)

# Normalizando os dados
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Treinando o modelo de regressão logística
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)
model.fit(X, y)

# Verificando classes treinadas
print("Classes treinadas pelo modelo:", model.classes_)

# Verificando as dimensões dos coeficientes e interceptos
print("Dimensões dos coeficientes do modelo:", model.coef_.shape)
print("Dimensões dos interceptos do modelo:", model.intercept_.shape)

# Função para criar uma explicação minimal usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=1e-6, timeout=60000):
    num_features = instance.shape[0]
    weights = model.coef_[0]
    intercept = model.intercept_[0]
    
    # Z3 Optimize
    opt = Optimize()
    opt.set("timeout", timeout)  # Adiciona limite de tempo (em milissegundos)
    
    # Variáveis Z3
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]
    
    # Função de decisão do modelo
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept
    
    # Adicionar restrições ao solver
    decision_target = decision_function(weights, intercept, instance, feature_selection)
    decision_other = -decision_function(weights, -intercept, instance, feature_selection)
    constraint = decision_target > decision_other + epsilon
    opt.add(constraint)
    
    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))

    # Check satisfiability and get the model if possible
    result = opt.check()
    
    if result == sat:
        m = opt.model()
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]
        explanation_features = [feature_names[i] for i in explanation]  # Nomes das características
        return result, explanation, explanation_features
    else:
        return result, None, None

# Exemplo de uso
instance = X[0]  # Exemplo de instância para explicar
target_class = model.predict([instance])[0]  # Classe prevista pelo modelo

print("Classe prevista pelo modelo para a instância:", target_class)

result, explanation, explanation_features = minimal_explanation(model, instance, target_class)
if explanation:
    print("Objective: Minimize selected features")
    print("Solver result:", result)
    print("Model found. Explanation (indices):", explanation)
    print("Explicação minimal (nomes das características):", explanation_features)
else:
    print("No solution found")


Número de amostras por classe após filtragem:
Classe 0: 6903
Classe 1: 7877
Classes treinadas pelo modelo: [0 1]
Dimensões dos coeficientes do modelo: (1, 784)
Dimensões dos interceptos do modelo: (1,)
Classe prevista pelo modelo para a instância: 0
No solution found


In [18]:
# MELHOR FUNCIONAL COM VÁRIAS OMODIFICAÇOES E SIMPLIFICAÇÕES NOS DADOS E NA REGRESSÃO LOGÍSTICA

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from z3 import *

# 1. Carregando o dataset MNIST
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data  # Matriz com os dados de pixels das imagens
y = mnist.target.astype(np.int8)  # Vetor com as labels (classes) das imagens

# 2. Selecionando apenas as classes 2, 4 e 7
mask = (y == 2) | (y == 4) | (y == 7)  # Criando uma máscara booleana para selecionar as classes desejadas
X = X[mask]  # Filtrando os dados para manter apenas as amostras das classes 2, 4 e 7
y = y[mask]  # Filtrando as labels para manter apenas as labels das classes 2, 4 e 7

# 3. Seleção de Pixels
k_features = 50  # Número de recursos (pixels) a serem selecionados
selector = SelectKBest(mutual_info_classif, k=k_features)  # Criando um objeto SelectKBest usando a informação dos pixels para selecionar recursos
X_selected = selector.fit_transform(X, y)  # Aplicando a seleção de recursos para selecionar os melhores k_features pixels
feature_names = [f'pixel_{i}' for i in range(X_selected.shape[1])]  # Criando nomes para os pixels selecionados

# 4. Normalizando os dados
scaler = StandardScaler()  # Criando um objeto StandardScaler para normalizar os dados
X_selected = scaler.fit_transform(X_selected)  # Normalizando os dados selecionados

# 5. Treinando o modelo de regressão logística
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)  # Criando um modelo de regressão logística
model.fit(X_selected, y)  # Treinando o modelo com os dados selecionados e normalizados

# 6. Verificando as classes treinadas
print("Classes treinadas pelo modelo:", model.classes_)

# 7. Verificando as dimensões dos coeficientes e interceptos
print("Dimensões dos coeficientes (pesos) do modelo:", model.coef_.shape)
print("Dimensões dos interceptos (viés) do modelo:", model.intercept_.shape)
# Dimensões dos coeficientes o primeiro numero é a quantidade de classes 
# Segundo número é a quantidade de pixels do MNIST para usar no modelo
# O vetor de interceptos possui um valor para cada classe, que representa 
# O valor da função de decisão (probabilidade) quando todas as features são zero.

# 8. Função para criar uma explicação minimal usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=0.2, timeout=60000):
    num_features = instance.shape[0]  # Número de recursos na instância (pixels selecionados)
    weights = model.coef_  # Pesos do modelo (coeficientes)
    intercepts = model.intercept_  # Interceptos do modelo
    
    # Z3 Optimize
    opt = Optimize()  # Criando um objeto Optimize para o Z3 Solver
    opt.set("timeout", timeout)  # Definindo um limite de tempo para o solver (60000 milissegundos = 1 minuto) para não demorar muito
    
    # Variáveis Z3 (representando a seleção de recursos)
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]  # Criando variáveis booleanas para cada recurso (pixel)

    # Função de decisão do modelo (usada para definir as restrições do Z3)
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept

    # Obter o índice da classe alvo
    target_index = np.where(model.classes_ == target_class)[0][0]  # Encontrando o índice da classe alvo

    # Adicionar restrições ao solver
    for i in range(len(model.classes_)):
        if i != target_index:  # Para cada classe diferente da classe alvo
            decision_target = decision_function(weights[target_index], intercepts[target_index], instance, feature_selection)  # Calculando a função de decisão para a classe alvo
            decision_other = decision_function(weights[i], intercepts[i], instance, feature_selection)  # Calculando a função de decisão para outra classe
            
            # Relaxamento das Restrições: Ajuste da folga (epsilon)
            constraint = decision_target > decision_other + epsilon  # Definindo a restrição para a função de decisão da classe alvo ser maior que a função de decisão das outras classes com uma margem (epsilon)
            opt.add(constraint)  # Adicionando a restrição ao solver

    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))  # Definindo a função de otimização para minimizar o número de recursos selecionados

    # Check satisfiability and get the model if possible
    result = opt.check()  # Verificando se o Z3 Solver encontrou uma solução satisfazendo as restrições

    if result == sat:  # Se o solver encontrou uma solução
        m = opt.model()  # Obtendo o modelo (interpretação) do solver
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]  # Identificando os recursos (pixels) selecionados na explicação minimal
        explanation_features = [feature_names[i] for i in explanation]  # Obtendo os nomes dos recursos (pixels) selecionados
        return result, explanation, explanation_features  # Retornando o resultado do solver, a lista de índices dos recursos selecionados e os nomes dos recursos selecionados
    else:  # Se o solver não encontrou uma solução
        return result, None, None  # Retornando None para a explicação

# 9. Exemplo de uso
instance = X_selected[0]  # Selecionando a primeira instância (imagem) do conjunto de dados
target_class = model.predict([instance])[0]  # Obtendo a classe prevista pelo modelo para a instância

print("Classe prevista pelo modelo para a instância:", target_class)

result, explanation, explanation_features = minimal_explanation(model, instance, target_class)
if explanation:
    print("Objective: Minimize selected features")
    print("Solver result:", result)
    print("Model found. Explanation (indices):", explanation)
    print("Explicação minimal (nomes das características):", explanation_features)
else:
    print("No solution found")

Classes treinadas pelo modelo: [2 4 7]
Dimensões dos coeficientes do modelo: (3, 50)
Dimensões dos interceptos do modelo: (3,)
Classe prevista pelo modelo para a instância: 4
Objective: Minimize selected features
Solver result: sat
Model found. Explanation (indices): [27, 46]
Explicação minimal (nomes das características): ['pixel_27', 'pixel_46']


Carregamento do Dataset MNIST: O código começa importando as bibliotecas necessárias, incluindo o fetch_openml para carregar o dataset MNIST. O dataset é dividido em duas partes: X (dados de pixels das imagens) e y (labels das imagens).
Seleção das Classes 2, 4 e 7: O código seleciona apenas as amostras das classes 2, 4 e 7 usando uma máscara booleana (mask) e filtra os dados X e as labels y.
Seleção de Recursos (Mutual Information): O código usa o SelectKBest para selecionar os melhores 50 pixels (features) com base na informação mútua (mutual_info_classif). Isso significa que ele escolhe os pixels que são mais informativos para a classificação das classes 2, 4 e 7. O resultado é armazenado em X_selected.
Normalização dos Dados: O código normaliza os dados selecionados (X_selected) usando o StandardScaler para garantir que todas as features tenham a mesma escala.
Treinamento do Modelo de Regressão Logística: O código treina um modelo de regressão logística usando os dados selecionados e normalizados.
Verificação das Classes Treinadas: O código exibe as classes que o modelo foi treinado a prever.
Verificação das Dimensões dos Coeficientes e Interceptos: O código exibe as dimensões dos coeficientes (pesos) e interceptos do modelo.
Função minimal_explanation:
Esta função usa o Z3 Solver para encontrar uma explicação minimal para a previsão do modelo.
Z3 Optimize: Um objeto Optimize do Z3 é criado para otimizar as restrições do solver.
Variáveis Z3: Variáveis booleanas (feature_selection) são criadas para representar a seleção de cada pixel (feature).
Função de Decisão: Uma função decision_function é definida para calcular a função de decisão do modelo, que representa a probabilidade de uma imagem pertencer a uma determinada classe.
Restrições: As restrições para o Z3 Solver são adicionadas usando a função de decisão. As restrições garantem que a função de decisão para a classe alvo seja maior que a função de decisão para as outras classes, com uma pequena margem (epsilon).
Otimização: A função minimize do Z3 é usada para minimizar o número de recursos selecionados, buscando a explicação minimal.
Verificação da Solução: O Z3 Solver verifica se existe uma solução satisfatória às restrições. Se houver, ele retorna a explicação minimal, que inclui a lista de índices dos recursos selecionados e seus nomes.
Exemplo de Uso:
O código seleciona uma instância (imagem) do conjunto de dados e usa o modelo para prever a classe.
A função minimal_explanation é chamada para encontrar a explicação minimal para a previsão.
Os resultados são impressos na tela, incluindo a explicação minimal.
Observações:
Ajuste os parâmetros k_features e epsilon para otimizar o desempenho do Z3 Solver.
Se você estiver tendo problemas para obter uma explicação minimal, considere tentar outras técnicas de simplificação do modelo, como reduzir ainda mais o número de recursos ou usar técnicas de regularização no modelo de regressão logística.
Para problemas complexos, você pode explorar bibliotecas de explicabilidade específicas para aprendizado de máquina, como SHAP e LIME.

In [30]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from z3 import *
import matplotlib.pyplot as plt

# 1. Carregando o dataset MNIST
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data  # Matriz com os dados de pixels das imagens
y = mnist.target.astype(np.int8)  # Vetor com as labels (classes) das imagens

# 2. Selecionando apenas as classes 2, 4 e 7
mask = (y == 2) | (y == 4) | (y == 7)  # Criando uma máscara booleana para selecionar as classes desejadas
X = X[mask]  # Filtrando os dados para manter apenas as amostras das classes 2, 4 e 7
y = y[mask]  # Filtrando as labels para manter apenas as labels das classes 2, 4 e 7

# 3. Seleção de Recursos (Mutual Information)
k_features = 50  # Número de recursos (pixels) a serem selecionados
selector = SelectKBest(mutual_info_classif, k=k_features)  # Criando um objeto SelectKBest usando a informação mútua para selecionar recursos
X_selected = selector.fit_transform(X, y)  # Aplicando a seleção de recursos para selecionar os melhores k_features pixels
feature_names = [f'pixel_{i}' for i in range(X_selected.shape[1])]  # Criando nomes para os pixels selecionados

# 4. Normalizando os dados
scaler = StandardScaler()  # Criando um objeto StandardScaler para normalizar os dados
X_selected = scaler.fit_transform(X_selected)  # Normalizando os dados selecionados

# 5. Treinando o modelo de regressão logística
model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)  # Criando um modelo de regressão logística
model.fit(X_selected, y)  # Treinando o modelo com os dados selecionados e normalizados

# 6. Verificando as classes treinadas
print("Classes treinadas pelo modelo:", model.classes_)

# 7. Verificando as dimensões dos coeficientes e interceptos
print("Dimensões dos coeficientes do modelo:", model.coef_.shape)
print("Dimensões dos interceptos do modelo:", model.intercept_.shape)

# 8. Função para criar uma explicação minimal usando Z3 Solver
def minimal_explanation(model, instance, target_class, epsilon=0.2, timeout=60000):
    num_features = instance.shape[0]  # Número de recursos na instância (pixels selecionados)
    weights = model.coef_  # Pesos do modelo (coeficientes)
    intercepts = model.intercept_  # Interceptos do modelo
    
    # Z3 Optimize
    opt = Optimize()  # Criando um objeto Optimize para o Z3 Solver
    opt.set("timeout", timeout)  # Definindo um limite de tempo para o solver (60000 milissegundos = 1 minuto)
    
    # Variáveis Z3 (representando a seleção de recursos)
    feature_selection = [Bool(f'f{i}') for i in range(num_features)]  # Criando variáveis booleanas para cada recurso (pixel)

    # Função de decisão do modelo (usada para definir as restrições do Z3)
    def decision_function(weights, intercept, instance, selected_features):
        return Sum([If(selected_features[i], instance[i] * weights[i], 0) for i in range(num_features)]) + intercept

    # Obter o índice da classe alvo
    target_index = np.where(model.classes_ == target_class)[0][0]  # Encontrando o índice da classe alvo

    # Adicionar restrições ao solver
    for i in range(len(model.classes_)):
        if i != target_index:  # Para cada classe diferente da classe alvo
            decision_target = decision_function(weights[target_index], intercepts[target_index], instance, feature_selection)  # Calculando a função de decisão para a classe alvo
            decision_other = decision_function(weights[i], intercepts[i], instance, feature_selection)  # Calculando a função de decisão para outra classe
            
            # Relaxamento das Restrições: Ajuste da folga (epsilon)
            constraint = decision_target > decision_other + epsilon  # Definindo a restrição para a função de decisão da classe alvo ser maior que a função de decisão das outras classes com uma margem (epsilon)
            opt.add(constraint)  # Adicionando a restrição ao solver

    # Minimizar o número de características selecionadas
    opt.minimize(Sum([If(f, 1, 0) for f in feature_selection]))  # Definindo a função de otimização para minimizar o número de recursos selecionados

    # Check satisfiability and get the model if possible
    result = opt.check()  # Verificando se o Z3 Solver encontrou uma solução satisfazendo as restrições

    if result == sat:  # Se o solver encontrou uma solução
        m = opt.model()  # Obtendo o modelo (interpretação) do solver
        explanation = [i for i in range(num_features) if m.evaluate(feature_selection[i])]  # Identificando os recursos (pixels) selecionados na explicação minimal
        explanation_features = [feature_names[i] for i in explanation]  # Obtendo os nomes dos recursos (pixels) selecionados
        return result, explanation, explanation_features  # Retornando o resultado do solver, a lista de índices dos recursos selecionados e os nomes dos recursos selecionados
    else:  # Se o solver não encontrou uma solução
        return result, None, None  # Retornando None para a explicação

# 9. Exemplo de uso
instance = X_selected[0]  # Selecionando a primeira instância (imagem) do conjunto de dados
target_class = model.predict([instance])[0]  # Obtendo a classe prevista pelo modelo para a instância

print("Classe prevista pelo modelo para a instância:", target_class)

result, explanation, explanation_features = minimal_explanation(model, instance, target_class)

# Pintando a imagem com a explicação minimal
if explanation:
    # 10. Obter a imagem original (usando X)
    image = X[0].reshape((28, 28))  # Pega a imagem original de X
    
    # 11. Criar uma cópia da imagem para pintar os pixels irrelevantes de vermelho
    image_with_explanation = image.copy()
    
    # 12. Transformar os índices da explicação minimal para os pixels originais
    explanation_original_indices = selector.inverse_transform(X_selected)[0][explanation]  # Transforma os indices para o X original

    # 13. Pintar os pixels irrelevantes de vermelho
    for i in range(image_with_explanation.shape[0]):
        for j in range(image_with_explanation.shape[1]):
            index = i * 28 + j
            if index not in explanation_original_indices:  # Use os indices originais
                image_with_explanation[i, j] = 255
    
    # 14. Plotar a imagem com a explicação minimal
    plt.figure(figsize=(4, 4))
    plt.imshow(image_with_explanation, cmap='gray')
    plt.title(f"Classe Prevista: {target_class}")
    plt.show()
    
    print("Objective: Minimize selected features")
    print("Solver result:", result)
    print("Model found. Explanation (indices):", explanation)
    print("Explicação minimal (nomes das características):", explanation_features)
else:
    print("No solution found")

Classes treinadas pelo modelo: [2 4 7]
Dimensões dos coeficientes do modelo: (3, 50)
Dimensões dos interceptos do modelo: (3,)
Classe prevista pelo modelo para a instância: 4


KeyError: 0